In [4]:
from GraphModels.model import CholPCorrModel

from sklearn.covariance import GraphicalLasso
from sklearn.model_selection import train_test_split

import numpy as np
import networkx as nx
from tqdm import tqdm
import warnings

from scipy.optimize import curve_fit

from collections import defaultdict

import optuna

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def f(x, a, b, c):
    return a * np.power(x, b) + c

params = curve_fit(f, np.arange(0.1, 1, 0.1), [0.94, 0.87, 0.85, 0.77, 0.74, 0.68, 0.64, 0.55, 0.4])[0]

In [7]:
def describe(arr):
    print('Mean:', np.mean(arr))
    print('Std:', np.std(arr))
    print('Median:', np.median(arr))

In [38]:
dim = 20
n_samples = 40
density = f(0.1, *params)
num_models = 20

def optimize(trial):
    alpha = trial.suggest_float('alpha', 1e-2, 1e3, step=0.01)

    val_scores = []
    for x in range(num_models):
        model = CholPCorrModel(dim, density)
        data = model.sample(n_samples)
        data_train, data_val = train_test_split(data, test_size=0.2)

        slv = GraphicalLasso(alpha, max_iter=1000, assume_centered=True)
        slv.fit(data_train)

        val_scores.append(slv.score(data_val))

    return np.mean(val_scores)

study = optuna.create_study(direction='maximize')
study.optimize(optimize, n_trials=1000, n_jobs=-1)


[I 2023-08-16 23:40:20,214] A new study created in memory with name: no-name-0e87b2ff-3345-4dfe-9b74-6671f6229970


[I 2023-08-16 23:40:22,038] Trial 1 finished with value: -33.09082966356439 and parameters: {'alpha': 351.2}. Best is trial 1 with value: -33.09082966356439.
[I 2023-08-16 23:40:22,046] Trial 6 finished with value: -33.004220755412675 and parameters: {'alpha': 341.16}. Best is trial 6 with value: -33.004220755412675.
[I 2023-08-16 23:40:22,318] Trial 2 finished with value: -32.39697056088131 and parameters: {'alpha': 184.35}. Best is trial 2 with value: -32.39697056088131.
[I 2023-08-16 23:40:22,348] Trial 0 finished with value: -31.820449818344844 and parameters: {'alpha': 257.62}. Best is trial 0 with value: -31.820449818344844.
[I 2023-08-16 23:40:22,378] Trial 3 finished with value: -33.268757311648415 and parameters: {'alpha': 567.42}. Best is trial 0 with value: -31.820449818344844.
[I 2023-08-16 23:40:22,402] Trial 5 finished with value: -32.81151284181087 and parameters: {'alpha': 917.3100000000001}. Best is trial 0 with value: -31.820449818344844.
[I 2023-08-16 23:40:22,582] T

In [41]:
study.best_params

{'alpha': 61.35}